In [1]:
import vk_api

Предварительный анализ скачанных жсон

In [2]:
jposts = vk_api.load_json('posts.json')

In [3]:
jposts[:5]

[{'attachments': [{'photo': {'access_key': '008739e87d330314d1',
     'album_id': -7,
     'date': 1493664165,
     'height': 750,
     'id': 456239019,
     'owner_id': 23704358,
     'photo_130': 'https://pp.userapi.com/c638720/v638720358/3224b/TSsh8ZWB2oA.jpg',
     'photo_604': 'https://pp.userapi.com/c638720/v638720358/3224c/ZUKVH-KDb30.jpg',
     'photo_75': 'https://pp.userapi.com/c638720/v638720358/3224a/A--QVaIuMAM.jpg',
     'photo_807': 'https://pp.userapi.com/c638720/v638720358/3224d/mN0bMSCNPmw.jpg',
     'post_id': 3560,
     'text': '',
     'width': 750},
    'type': 'photo'}],
  'comments': {'can_post': 0, 'count': 0, 'groups_can_post': True},
  'date': 1493664166,
  'from_id': 23704358,
  'id': 3560,
  'likes': {'can_like': 1, 'can_publish': 1, 'count': 9, 'user_likes': 0},
  'owner_id': 23704358,
  'post_source': {'platform': 'instagram',
   'type': 'api',
   'url': 'https://www.instagram.com/p/BTgQBCqFgOO/'},
  'post_type': 'post',
  'reposts': {'count': 0, 'user_re

In [4]:
jposts[0].keys()

dict_keys(['attachments', 'comments', 'date', 'from_id', 'id', 'likes', 'owner_id', 'post_source', 'post_type', 'reposts', 'text', 'views'])

In [5]:
jposts[1013].keys()

dict_keys(['comments', 'copy_history', 'date', 'from_id', 'id', 'likes', 'owner_id', 'post_source', 'post_type', 'reposts', 'text'])

Трансформация json в dataframe реализовано в модуле vk_api
в двух методах - read_posts и read_users

In [2]:
post_df = vk_api.read_posts()
post_df.columns

Index(['owner_id', 'text', 'comments_count', 'likes_count', 'reposts_count',
       'date', 'post_type', 'views_count', 'copy_history_text',
       'copy_history_post_type', 'attachments_video_count',
       'attachments_photo_count', 'attachments_audio_count'],
      dtype='object')

In [3]:
user_df = vk_api.read_users()
print(user_df.columns)
user_df = user_df[['id','about', 'byear', 'books', 'activities', 'graduation',  'interests','movies', 'music', 'sex', 'relation' ]]

Index(['about', 'activities', 'bdate', 'books', 'byear', 'country_id',
       'country_title', 'education_form', 'education_status', 'faculty',
       'faculty_name', 'first_name', 'graduation', 'home_phone', 'home_town',
       'id', 'interests', 'last_name', 'mobile_phone', 'movies', 'music',
       'personal_alcohol', 'personal_inspired_by', 'personal_langs_0',
       'personal_langs_1', 'personal_langs_10', 'personal_langs_11',
       'personal_langs_12', 'personal_langs_13', 'personal_langs_14',
       'personal_langs_15', 'personal_langs_16', 'personal_langs_17',
       'personal_langs_18', 'personal_langs_19', 'personal_langs_2',
       'personal_langs_20', 'personal_langs_21', 'personal_langs_22',
       'personal_langs_23', 'personal_langs_24', 'personal_langs_25',
       'personal_langs_26', 'personal_langs_27', 'personal_langs_28',
       'personal_langs_29', 'personal_langs_3', 'personal_langs_4',
       'personal_langs_5', 'personal_langs_6', 'personal_langs_7',
       'pe

Итого определились с полями у пользователей и в постах

In [4]:
for col in post_df.columns:
    print(col)

owner_id
text
comments_count
likes_count
reposts_count
date
post_type
views_count
copy_history_text
copy_history_post_type
attachments_video_count
attachments_photo_count
attachments_audio_count


In [5]:
for col in user_df.columns:
    print(col)    

id
about
byear
books
activities
graduation
interests
movies
music
sex
relation


Заполним отсутствующий текст текстом из репоста, а если вообще нет текстат ни там ни там, заполним типом поста - есть варианты можно заполнить типа вложений

In [6]:
post_df.loc[post_df['text']=='','text'] = post_df['copy_history_text'].fillna(post_df['post_type'])

Удалим дубликаты

In [7]:
post_df = post_df.drop_duplicates()
post_df.count()

owner_id                   29929
text                       29929
comments_count             29929
likes_count                29929
reposts_count              29929
date                       29929
post_type                  29929
views_count                29929
copy_history_text          22236
copy_history_post_type     22236
attachments_video_count     6035
attachments_photo_count     6035
attachments_audio_count     6035
dtype: int64

Смержим с пользователями

In [8]:
post_df = post_df.merge(user_df, how="inner", left_on="owner_id",right_on='id')

Пермешаем посты  и для начала нам достаточно года рождения и текста, с них и начнем

In [9]:
#post_df
from sklearn.utils import shuffle
df = shuffle(post_df[['byear', 'text']])

In [10]:
df

,byear,text
27699,1989,
11124,1996,
10358,1990,#микростатья@microbiology720\n\nЧасть 2.\n\nВт...
19929,1985,"3 сентября, среда\n\nРыбы не надо тупить!\nУда..."
20646,1911,Природа проверяет свои творческие способности
29200,1986,Супер профессионал! Рекомендую
10308,1990,
10540,1993,
11408,1989,
793,1993,


Разобьем на train и test

In [19]:
df_train = df[0:10000]
df_test = df[10001:20000]

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
x = count_vect.fit_transform(df_train['text']).todense()

In [13]:
x.shape

(10000, 67723)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(df_train['text'])

In [15]:
features.shape

(10000, 67723)

In [31]:
vectorizer.get_feature_names()[10000:11000]

['бывать',
 'бываю',
 'бывают',
 'бывшая',
 'бывшего',
 'бывшей',
 'бывшие',
 'бывший',
 'бывшим',
 'бывшими',
 'бывшимпозвонин',
 'бывших',
 'быдгощ',
 'быдло',
 'бык',
 'быки',
 'быков',
 'быкова',
 'был',
 'былo',
 'была',
 'были',
 'былинно',
 'былины',
 'было',
 'былое',
 'былую',
 'былые',
 'былым',
 'быль',
 'былі',
 'былა',
 'былო',
 'быстpo',
 'быстрая',
 'быстрее',
 'быстрей',
 'быстренько',
 'быстро',
 'быстрого',
 'быстродействующие',
 'быстродействующий',
 'быстродействующих',
 'быстрое',
 'быстрозастывающий',
 'быстрой',
 'быстрому',
 'быстрорастворимого',
 'быстрорастворимый',
 'быстрорастущее',
 'быстрота',
 'быстроты',
 'быструмгель',
 'быструю',
 'быстрые',
 'быстрый',
 'быстрым',
 'быстрых',
 'быт',
 'быта',
 'быте',
 'бытие',
 'бытия',
 'бытовая',
 'бытового',
 'бытовой',
 'бытовухи',
 'бытовую',
 'бытовые',
 'бытовых',
 'быту',
 'бытует',
 'быть',
 'быць',
 'бычок',
 'быў',
 'быვაюტ',
 'быლ',
 'быლო',
 'быტь',
 'бьет',
 'бьется',
 'бьорн',
 'бьюсь',
 'бьют',
 'бьют

In [16]:
vectorizer.stop_words_

set()

Попробуем построить модель 

In [17]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import time

def randomized_cv(model, param_grid, x_train, y_train):
    grid_search = RandomizedSearchCV(model, param_grid, cv=5, scoring='accuracy', n_iter=5)
    t_start = time.time()
    grid_search.fit(x_train, y_train)
    t_end = time.time()
    print('model {} best accuracy score is {}'.format(model.__class__.__name__, grid_search.best_score_))
    print('time for training is {} seconds'.format(t_end - t_start))
    return grid_search.best_estimator_

In [18]:
param_grid = {'alpha':[0.8, 0.09, 0.1, 1.1, 1.2]}
model = MultinomialNB()
X_train = features.todense()
y_train = df_train['byear']
best_model = randomized_cv(model, param_grid, X_train, y_train)

model MultinomialNB best accuracy score is 0.2055
time for training is 5594.385264873505 seconds


In [20]:
best_model

MultinomialNB(alpha=0.09, class_prior=None, fit_prior=True)

Модель не очень удачная
посмотрим сколько у нас классов

In [45]:
y_train.describe()

count     10000
unique       30
top        1992
freq        987
Name: byear, dtype: object

In [46]:
param_grid = {'alpha':[0.8, 0.09, 0.1, 1.1, 1.2]}
model = BernoulliNB()
best_model = randomized_cv(model, param_grid, X_train, y_train)
best_model

model BernoulliNB best accuracy score is 0.1481
time for training is 8.258578538894653 seconds


BernoulliNB(alpha=0.09, binarize=0.0, class_prior=None, fit_prior=True)

In [48]:
model = GaussianNB()
model.fit(X_train.todense(), y_train)
y_pred = model.predict(X_train)
print(accuracy_score(y_train, y_pred))

MemoryError: 